# Building a Neural Network from Scratch

http://jonathanweisberg.org/post/A%20Neural%20Network%20from%20Scratch%20-%20Part%201/


In this post we’re going to build a neural network from scratch. We’ll train it to recognize hand-written digits, using the famous MNIST data set.

We’ll use just basic Python with NumPy to build our network (no high-level stuff like Keras or TensorFlow). We will dip into scikit-learn, but only to get the MNIST data and to assess our model once its built.

---

#  (Fashion MNIST)
## data:  70000

In [1]:
import time
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [3]:
path = '/Users/reshamashaikh/ds/my_repos/pytorch_work/data_fashion/'
#path = '/Users/reshamashaikh/ds/my_repos/fashion_mnist/raw'

In [8]:
!ls -alt {path}

total 171304
drwxr-xr-x  22 reshamashaikh  staff       704 Sep  9 16:47 ..
-rw-r--r--@  1 reshamashaikh  staff      6148 Aug 31 22:59 .DS_Store
drwxr-xr-x  11 reshamashaikh  staff       352 Aug 31 22:59 .
-rw-r--r--   1 reshamashaikh  staff     10008 Aug 31 22:59 t10k-labels-idx1-ubyte
-rw-r--r--   1 reshamashaikh  staff     60008 Aug 31 22:58 train-labels-idx1-ubyte
-rw-r--r--   1 reshamashaikh  staff  47040016 Aug 31 22:58 train-images-idx3-ubyte
-rw-r--r--   1 reshamashaikh  staff   7840016 Aug 31 22:58 t10k-images-idx3-ubyte
-rw-r--r--@  1 reshamashaikh  staff      5148 Aug 31  2017 t10k-labels-idx1-ubyte.gz
-rw-r--r--@  1 reshamashaikh  staff   4422102 Aug 31  2017 t10k-images-idx3-ubyte.gz
-rw-r--r--@  1 reshamashaikh  staff     29515 Aug 31  2017 train-labels-idx1-ubyte.gz
-rw-r--r--@  1 reshamashaikh  staff  26421880 Aug 31  2017 train-images-idx3-ubyte.gz


In [9]:
X_train, Y_train = load_mnist(path, kind='train')
X_test, Y_test = load_mnist(path, kind='t10k')

In [10]:
print(type(X_train))
print(X_train.shape)

print(type(Y_train))
print(Y_train.shape)

print(type(X_test))
print(X_test.shape)

print(type(Y_test))
print(Y_test.shape)

<class 'numpy.ndarray'>
(60000, 784)
<class 'numpy.ndarray'>
(60000,)
<class 'numpy.ndarray'>
(10000, 784)
<class 'numpy.ndarray'>
(10000,)


In [ ]:
t0 = time.time()

In [ ]:
# from sklearn.datasets import fetch_mldata

# mnist = fetch_mldata('MNIST original')
# X, y = mnist["data"], mnist["target"]

In [ ]:
# We’ll normalize the data to keep our gradients manageable:

#X = X / 255

In [ ]:
# Then we’ll one-hot encode MNIST’s labels, to get a 10 x 70,000 array.

# digits = 10
# examples = y.shape[0]

# y = y.reshape(1, examples)

# Y_new = np.eye(digits)[y.astype('int32')]
# Y_new = Y_new.T.reshape(digits, examples)

In [ ]:
# Then we re-split, re-shape, and re-shuffle our training set:

m = 60000
m_test = 10000

# X_train, X_test = X[:m].T, X[m:].T
# Y_train, Y_test = Y_new[:,:m], Y_new[:,m:]

shuffle_index = np.random.permutation(m)
X_train, Y_train = X_train[:, shuffle_index], Y_train[:, shuffle_index]

In [ ]:
# A quick check that things are as they should be:

i = 12
plt.imshow(X_train[:,i].reshape(28,28), cmap = matplotlib.cm.binary)
plt.axis("off")
plt.show()
Y_train[:,i]

In [ ]:
#So let’s define:

def compute_multiclass_loss(Y, Y_hat):

    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    L = -(1/m) * L_sum

    return L

In [ ]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

### one hidden layer

In [ ]:
# 4.5 Build & Train

n_x = X_train.shape[0]
n_h = 64
learning_rate = 1

W1 = np.random.randn(n_h, n_x)
b1 = np.zeros((n_h, 1))
W2 = np.random.randn(digits, n_h)
b2 = np.zeros((digits, 1))

X = X_train
Y = Y_train

for i in range(2000):

    Z1 = np.matmul(W1,X) + b1
    A1 = sigmoid(Z1)
    Z2 = np.matmul(W2,A1) + b2
    A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

    cost = compute_multiclass_loss(Y, A2)

    dZ2 = A2-Y
    dW2 = (1./m) * np.matmul(dZ2, A1.T)
    db2 = (1./m) * np.sum(dZ2, axis=1, keepdims=True)

    dA1 = np.matmul(W2.T, dZ2)
    dZ1 = dA1 * sigmoid(Z1) * (1 - sigmoid(Z1))
    dW1 = (1./m) * np.matmul(dZ1, X.T)
    db1 = (1./m) * np.sum(dZ1, axis=1, keepdims=True)

    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1

    if (i % 100 == 0):
        print("Epoch", i, "cost: ", cost)

print("Final cost:", cost)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Let’s see how we did:

Z1 = np.matmul(W1, X_test) + b1
A1 = sigmoid(Z1)
Z2 = np.matmul(W2, A1) + b2
A2 = np.exp(Z2) / np.sum(np.exp(Z2), axis=0)

predictions = np.argmax(A2, axis=0)
labels = np.argmax(Y_test, axis=0)

index = 0
misclassifiedIndexes = []
for label, predict in zip(labels, predictions):
    if label != predict: 
        misclassifiedIndexes.append(index)
    index +=1

print(confusion_matrix(predictions, labels))
print(classification_report(predictions, labels))

In [ ]:
print(len(misclassifiedIndexes))

In [ ]:
print("Accuracy:  ", 1 - (818/10000))